### Nobels by City Map

In [64]:
!pip install geopy
!pip install altair
!pip install vega_datasets
!pip install vl-convert-python



import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

import altair as alt
from vega_datasets import data

import pickle as pkl
import os

     |████████████████████████████████| 21.0 MB 71 kB/s eta 0:00:012


Loading the data from the xlsx into a dataframe and preserving only the columns we need.

In [7]:
df = pd.read_excel('cleanNobel.xlsx')
df.head(3)

id  year category  share  \
0    1  1901  physics      1   
1   10  1906  physics      1   
2  100  1974  physics      2   

                                          motivation                     name  \
0  in recognition of the extraordinary services h...        Munich University   
1  in recognition of the great merits of his theo...  University of Cambridge   
2  for their pioneering research in radio astroph...  University of Cambridge   

        city         country overallMotivation       firstname  ...  \
0     Munich         Germany               NaN  Wilhelm Conrad  ...   
1  Cambridge  United Kingdom               NaN            J.J.  ...   
2  Cambridge  United Kingdom               NaN          Martin  ...   

                  born                 died            bornCountry  \
0           1845-03-27  1923-02-10 00:00:00  Prussia (now Germany)   
1           1856-12-18  1940-08-30 00:00:00         United Kingdom   
2  1918-09-27 00:00:00  1984-10-14 00:00:00         United Kingdom   

  bornCountryCode                bornCity     diedCountry diedCountryCode  \
0              DE  Lennep (now Remscheid)         Germany              DE   
1              GB           Cheetham Hill  United Kingdom              GB   
2              GB                Brighton  United Kingdom              GB   

    diedCity gender       _merge  
0     Munich   male  matched (3)  
1  Cambridge   male  matched (3)  
2  Cambridge   male  matched (3)  

[3 rows x 21 columns]

Using the Nominatim geocoder to get the coordinates of the cities.

In [8]:
geolocator = Nominatim(user_agent="nobels")

cached_locations = {}
if os.path.exists('cached_locations.pkl'):
    with open('cached_locations.pkl', 'rb') as f:
        cached_locations = pkl.load(f)

def get_location(city_Country):
    global location_counter
    if city_Country not in cached_locations:
        cached_locations[city_Country] = geolocator.geocode(city_Country)
    if(location_counter % 100 == 0):
         print(f'{location_counter}/1020 Processed')
    location_counter += 1
    return cached_locations[city_Country]

In [9]:
location_counter = 0
df['city_Country'] = df['bornCity'] + ', ' + df['bornCountry']
df['location'] = df['city_Country'].apply(get_location)

0/1020 Processed
100/1020 Processed
200/1020 Processed
300/1020 Processed
400/1020 Processed
500/1020 Processed
600/1020 Processed
700/1020 Processed
800/1020 Processed
900/1020 Processed
1000/1020 Processed


this loads the returned geolocator objects into the column - useful but not particularly readable.
We can then extract the lat_lon from the geolocator objects and put them into separate columns.

In [10]:
df['lat'] = df['location'].apply(lambda x: x.latitude if x != None else None)
df['lon'] = df['location'].apply(lambda x: x.longitude if x != None else None)

In [11]:
# Count None in lat and lon
unmatched = df[df['lat'].isnull()][['city_Country', 'bornCity', 'bornCountry']].groupby(by=['city_Country', 'bornCity', 'bornCountry']).first()
unmatched.reset_index(inplace=True)
print(f' {len(unmatched)} were not found: {unmatched["city_Country"].unique()} ')


 116 were not found: ['Aldea Chimel, Guatemala' 'Aurich, East Friesland (now Germany)'
 'Bad Salzbrunn, Prussia (now Poland)'
 'Baku, Russian Empire (now Azerbaijan)' 'Berlin, Prussia (now Germany)'
 'Bnin (now Kórnik), Poland' 'Bombay, British India (now India)'
 'Breslau (now Wroclaw), Germany (now Poland)'
 'Breslau (now Wroclaw), Prussia (now Poland)'
 'Brest Litovsk, Russian Empire (now Belarus)'
 'Briesen, Prussia (now Germany)'
 'Buczacz (now Buchach), Austria-Hungary (now Ukraine)'
 'Budapest, Austria-Hungary (now Hungary)'
 'Bukavu, Belgian Congo (now Democratic Republic of the Congo)'
 'Bulawayo, Southern Rhodesia (now Zimbabwe)'
 'Casteldàwson, Northern Ireland'
 'Castries, British West Indies (now Saint Lucia)'
 'Chittagong, British India (now Bangladesh)'
 'Clausthal (now Clausthal-Zellerfeld), Germany'
 'Constantine, French Algeria (now Algeria)' 'Dabrovica, Poland'
 'Danzig (now Gdansk), Free City of Danzig (now Poland)'
 'Dolac, Bosnia (now Bosnia and Herzegovina)'
 'Eu

For the 116 not found, we can manually clean the city_country column for some
and for others we can try and batch it by taking the countries in parentheses and try again.

e.g. Moscow, USSR (now Russia) -> Moscow, Russia

In [12]:
def current_name(old):
    if '(now ' in old:
        return old[old.index("(now ")+5:-1]
    else:
        return old

In [13]:
new_names = [current_name(un['bornCity']) + ", " + current_name(un['bornCountry']) for un in unmatched.to_dict(orient='records')]
new_names = dict(zip(unmatched.city_Country.values, new_names))

In [16]:
substitutions = {
    'Frankfurt-on-the-Main, Germany' : 'Frankfurt, Germany',
    'Moscow, USSR (now Russia)' : 'Moscow, Russia',
    'Strasbourg, Germany (now France)' : 'Strasbourg, France',
    'Prague, Czechoslovakia (now Czech Republic)' : 'Prague, Czech Republic',
    'Altenberg; Grünau im Almtal, Austria' : 'Altenberg, Austria',
    'Harwell, Berkshire, United Kingdom' : 'Harwell, United Kingdom',
    'Jhang Maghiāna, Pakistan' : 'Jhang, Pakistan',
    'Mount Verno, NY, USA' : 'Mount Vernon, New York, USA',
    'Dabrovica, Poland' : 'Lublin, Poland',
    'Parusnoye, Russia' : 'Parusnoe, Russia'  ,
    'Goldschmieden, near Breslau, Poland' :  'Breslau, Poland' ,
    'Rufford, near Chesterfield, United Kingdom' :  'Chesterfield, United Kingdom' ,
    'Zloczov, Ukraine' :  'Zolochiv, Ukraine' ,
    'Zelvas, Lithuania' :  'Zelva, Lithuania' ,
    'Thorshavn, Faroe Islands (Denmark)' : 'Thorshavn, Faroe Islands'  ,
    'Lasi, Romania' : 'Iași, Romania'  ,
    'Tardebigg, United Kingdom' :  'Tardebigge, United Kingdom' ,
    'Mit Abu al-Kawm, Egypt' :  'Tanta, Egypt' ,
    'Aldea Chimel, Guatemala' :  'Cahabon, Guatemala' ,
    'Vishneva, Belarus' :  'Minsk, Belarus' ,
    'Wailacama, East Timor' :  'Dili, East Timor', # The only references to Wailacama are in relation to the laureate - he later served in Dili though
    'Casteldàwson, Northern Ireland' : 'Londonderry, Northern Ireland'  ,
    'Hofei, Anhwei, China' :  'Hefei, China' ,
    'Langford Grove, Maldon, Essex, United Kingdom' :  'Maldon, United Kingdom' ,
    'Nitzkydorf, Banat, Romania' : 'Banat, Romania'  ,
    'Kibbutz Sde-Nahum, Israel' : 'Sde Nahum, Israel'  ,

} 
substitutions = {**new_names, **substitutions}

df['city_Country'] = df['city_Country'].replace(substitutions)

location_counter = 0
df['location'] = df['city_Country'].apply(get_location)
df['lat'] = df['location'].apply(lambda x: x.latitude if x != None else None)
df['lon'] = df['location'].apply(lambda x: x.longitude if x != None else None)

unmatched = df[df['lat'].isnull()]['city_Country'].unique()
print(f' {len(unmatched)} were not found')


0/1020 Processed
100/1020 Processed
200/1020 Processed
300/1020 Processed
400/1020 Processed
500/1020 Processed
600/1020 Processed
700/1020 Processed
800/1020 Processed
900/1020 Processed
1000/1020 Processed
 0 were not found


In [17]:

for un in unmatched:
    print(f"'{un}' :   ," )

Adding a little jitter:
Some cities have many Nobel receipients. For these, only one point is displayed on the map which can be misleading. To avoid this, we add a little jitter to the coordinates of the cities with many Nobel receipients. Applying jitter with its own column keeps the jitter constant across life and death

In [18]:
df['jitter'] = df['lat'].apply(lambda x: np.random.normal(0, 0.5)*0.1 if x != None else None)
df['lat_j'] = df['lat'] + df['jitter']
df['lon_j'] = df['lon'] + df['jitter']

and finally:
-  adding a full name column
-  capitalising category names

In [19]:
df['laureate'] = df['firstname'] + ' ' + df['surname']
# capitalise cateogry
df['category'] = df['category'].str.capitalize()

also getting death place coordinates for the laureates

In [20]:
df['death_city_Country'] = df['diedCity'] + ', ' + df['diedCountry']
location_counter = 0
df['death_location'] = df['death_city_Country'].apply(lambda x: get_location(x) if x != 'nan' else None)
df['death_lat'] = df['death_location'].apply(lambda x: x.latitude if x != None else None)
df['death_lon'] = df['death_location'].apply(lambda x: x.longitude if x != None else None)

0/1020 Processed
100/1020 Processed
200/1020 Processed
300/1020 Processed
400/1020 Processed
500/1020 Processed
600/1020 Processed
700/1020 Processed
800/1020 Processed
900/1020 Processed
1000/1020 Processed


and again checking for unmatched:

In [22]:
# Count None in lat and lon
unmatched = df[df['death_lat'].isnull()][['death_city_Country', 'diedCity', 'diedCountry']].groupby(by=['death_city_Country', 'diedCity', 'diedCountry']).first()
unmatched.reset_index(inplace=True)
print(f' {len(unmatched)} were not found: {unmatched["death_city_Country"].unique()} ')

 21 were not found: ['Agnetendorf (now Jagniatków), Germany (now Poland)'
 'Belgrade, Yugoslavia (now Serbia)' 'Berlin, East Germany (now Germany)'
 'Berlin, West Germany (now Germany)'
 'Bornheim-Merten, West Germany (now Germany)'
 'Burgberg, West Germany (now Germany)'
 'Cologne, West Germany (now Germany)'
 'Freiburg im Breisgau, West Germany (now Germany)'
 'Göttingen, West Germany (now Germany)'
 'Heidelberg, West Germany (now Germany)'
 'Kiel, West Germany (now Germany)' 'Moscow, USSR (now Russia)'
 'Mount Verno, NY, USA' 'Munich, West Germany (now Germany)'
 'Mülheim, West Germany (now Germany)'
 'Ndola, Northern Rhodesia (now Zambia)'
 'Prague, Czechoslovakia (now Czech Republic)'
 'Schopfheim, West Germany (now Germany)'
 'Traunstein, West Germany (now Germany)' 'Veshenskaya, USSR (now Russia)'
 'West Berlin, West Germany (now Germany)'] 


In [23]:
new_names = [current_name(un['diedCity']) + ", " + current_name(un['diedCountry']) for un in unmatched.to_dict(orient='records')]
new_names = dict(zip(unmatched.death_city_Country.values, new_names))

substitutions = {**new_names, **substitutions}

df['death_city_Country'] = df['death_city_Country'].replace(substitutions)

location_counter = 0
df['death_location'] = df['death_city_Country'].apply(lambda x: get_location(x) if str(x) != 'nan' else None)
df['death_lat'] = df['death_location'].apply(lambda x: x.latitude if x != None else None)
df['death_lon'] = df['death_location'].apply(lambda x: x.longitude if x != None else None)

unmatched = df[df['lat'].isnull()]['death_city_Country'].unique()
print(f' {len(unmatched)} were not found')


0/1020 Processed
100/1020 Processed
200/1020 Processed
300/1020 Processed
400/1020 Processed
500/1020 Processed
600/1020 Processed
 0 were not found


and adding a little jitter to the laureate death coordinates

In [24]:
df['death_lat_j'] = df['death_lat'] + df['jitter']
df['death_lon_j'] = df['death_lon'] + df['jitter']

and finally adding an id that's unique to the people (not just recepeints - which have duplicates for group winners)

In [25]:
df['individual_id'] = df.index

and fixing some of the dates:

In [42]:
df.loc[df['born'].apply(lambda x: type(x) == str), 'born'] = df.loc[df['born'].apply(lambda x: type(x) == str), 'born'].apply(lambda x: pd.to_datetime(x, errors='coerce'))
df.loc[df['died'].apply(lambda x: type(x) == str), 'died'] = df.loc[df['died'].apply(lambda x: type(x) == str), 'died'].apply(lambda x: pd.to_datetime(x, errors='coerce'))

and let's save the data to a csv file and to a json for d3:

In [43]:
df.drop(columns=['location', 'death_location']).to_csv('nobel_laureates_lat_lon.csv', index=False)
df.drop(columns=['location', 'death_location']).to_json('nobel_laureates_lat_lon.json', orient='records')

In [44]:
with open('cached_locations.pkl', 'wb') as f:
        pkl.dump(cached_locations, f)

### Let's chart it:

First a helper function for charting:

In [45]:
def nobelMap(o):
    # Plot lat lon columns from pandas df as points on world map using altair
    world = alt.topo_feature(data.world_110m.url, feature='countries')

    background = alt.Chart(world).mark_geoshape(
        fill=o['country_fill'],
        stroke=o['country_stroke']
    ).transform_filter(
        "datum.id != '010'"
    ).properties(
        width=o['width'],
    ).project('naturalEarth1', clipExtent=o['clipExtent'])

    filtered_df = df[df.category.isin(o['prizes'])]

    nobels = alt.Chart(
        filtered_df[['id', 'category', 'laureate',  'lat', 'lon', 'lat_j', 'year', 'lon_j', 'city_Country']],
        title="Birthplaces of Nobel Laureates").mark_circle().encode(
        size=alt.value(12),
        longitude='lon_j:Q',
        latitude='lat_j:Q',
        tooltip=['laureate', 'category', 'year', {
            'title': 'Birthplace', 'field': 'city_Country'}],
    ).project('naturalEarth1', clipExtent=o['clipExtent'])

    if not o['jitter']:
        nobels = nobels.encode(
            longitude='lon:Q',
            latitude='lat:Q',
        )


    scale=alt.Scale(domain=['Physics', 'Chemistry', 'Peace', 'Medicine', 'Literature', 'Economics'])

    if(o['color_by_category']):
        nobels = nobels.encode(
            color=alt.Color(
                'category:N', 
                scale=scale, legend={"values" : o['prizes'],
})
        )


    chart = background + nobels

    chart = chart.configure_title(
        fontSize=16,
        font='Times New Roman',
        fontWeight='normal',
        anchor='start',
        dx=0.081*o['width'],
        orient='top'
    )

    chart = chart.configure_legend(
        labelFont='Times New Roman',
        title=None,
        titleAnchor='start',
        direction='horizontal',
        legendX=0.081*o['width'],
        orient='none'
    )
    return(chart)


Defining an dict, o, for easily changing options

In [47]:
o = {
    'color_by_category' : True,
    'jitter': [0],
    'prizes' : list(['Physics', 'Chemistry', 'Peace', 'Medicine', 'Literature', 'Economics']),
    'title' : 'Birthplaces of Nobel Laureates',
    'country_fill' : 'white',
    'country_stroke' : 'darkgrey',
    'width' : 600,
    'height' : 400,
    'font' : 'Times New Roman',
    'projection' : 'naturalEarth1',
    'color_scheme' : 'inferno',
    'projection_border' : [[0.0, 0], [1,1]]
}

o['clipExtent'] = (np.array(o['projection_border']) * np.array([[o['width'], o['height']], [o['width'], o['height']]])).tolist()

nobelMap(o)


alt.LayerChart(...)

### Different Options
1. A map with points coloured by prize:

In [69]:
map_main = nobelMap(o)
map_main.save('maps_html/birthplaces_allprizes.html')
map_main.save('maps_json/birthplaces_allprizes.json')
map_main.save('maps_png/birthplaces_allprizes.png', scale_factor=3.0)
map_main.save('maps_svg/birthplaces_allprizes.svg')

2. Same Colour for all prizes

In [70]:
o1 = o.copy()
o1['color_by_category'] = False
one_colour_map = nobelMap(o1)
one_colour_map.save('maps_html/birthplaces_onecolour.html')
one_colour_map.save('maps_json/birthplaces_onecolour.json')
one_colour_map.save('maps_png/birthplaces_onecolour.png', scale_factor=3.0)
one_colour_map.save('maps_svg/birthplaces_onecolour.svg')

3. Separate maps for each prize

In [71]:
for prize in ['Physics', 'Chemistry', 'Peace', 'Medicine', 'Literature', 'Economics']:
    cur_map = nobelMap({**o, 'prizes':[prize]})
    cur_map.save(f'maps_html/birthplaces_{prize}.html')
    cur_map.save(f'maps_json/birthplaces_{prize}.json')
    cur_map.save(f'maps_png/birthplaces_{prize}.png', scale_factor=3.0)
    cur_map.save(f'maps_svg/birthplaces_{prize}.svg')
    cur_map.display()

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)